# Tutorial 11 - Introduction to Reinforcement Learning: Bandit Problems

*Written and revised by Jozsef Arato, Mengfan Zhang, Dominik Pegler*  
Computational Cognition Course, University of Vienna  
https://github.com/univiemops/tewa1-computational-cognition

---
# This week's lab:

This week's course content focuses on reinforcement learning, specifically the simpler case with only one state: **bandit problems**. These problems are fundamental to understanding reinforcement learning and have strong ties to psychology. For instance, they mirror the psychological processes behind conditioning and how organisms **learn from rewards**. In multiple one-armed-bandit problems, we're faced with multiple actions, each promising uncertain rewards. The tricky part is the **exploration-exploitation dilemma** – deciding whether to try new options (explore) or stick with rewarding past actions (exploit). In this tutorial we will simulate a human agent who tries to win the most money while playing on three bandits for a number of trials.

**Learning Goals:**

- Defining reward
- Understanding action-values $Q$
- Implementing strategies to tackle the explore-exploit-tradeoff
- Understanding stationarity vs. non-stationarity

**Reference:**
- Sutton, R. S., & Barto, A. G. (2018). Reinforcement learning: An
  introduction, 2nd ed. : The MIT Press.


---
<img src="https://people.stfx.ca/jdelamer/courses/csci-531/_images/multi-armed-bandit-slots-est-value.drawio.png" alt="bandit-problem" style="width:800px; background:white">

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## 1. The Environment

### 1.1. Initial Parameters

In reinforcement learning, the environment is one of the two main components, the other being the agent itself. It defines and provides the different states, the possible actions that can be taken, and the rewards associated with these actions. This setup allows the agent to interact with and learn from the environment by making decisions and observing the outcomes.

In [ ]:
n_bandit = 3  # number of bandits
actions = list(range(n_bandit))

### 1.2. The Reward Function

The reward function is part of the environment. Let's define it here as `compute_reward()`. E.g., each bandit should give a reward of 1, but the probabilities that the bandits give the reward are different. Write a function that takes in an action (the chosen bandit, e.g., 0 for the first bandit) and returns the reward based on the probabilities. The reward probabilities are 0.8 for bandit 0, 0.2 for bandit 1 and 0.4 for bandit 2. You can use `np.random.binomial()` to determine whether a reward is returned based on a probability.

In [ ]:
reward_probs = [0.8, 0.2, 0.4]

# YOUR CODE HERE
# --------------


# --------------

In [ ]:
# TEST CELL

test_rewards = np.zeros(3)
for action in range(n_bandit):
    for i in range(100_000):
        test_rewards[action] = test_rewards[action] + 1 / (i + 1) * (
            compute_reward(action, reward_probs) - test_rewards[action]
        )

correct = (np.array_equal(np.round(test_rewards, 1), [0.8, 0.2, 0.4])) and (
    compute_reward(0, reward_probs) in [0, 1]
)

if correct:
    print("Well done! Your reward function works as expected.")
else:
    raise Exception("Uh-oh! Check your code again.")

Now let's test our reward function with a simple simulation to see what reward each bandit gives us on average. If we did it right, it should converge to our initial reward probabilities.

In [ ]:
n_sim = 300

test_rewards = np.zeros((3, n_sim))

for action in range(n_bandit):
    for i in range(0, n_sim):
        if i == 0:
            test_rewards[action][i] = test_rewards[action][i] + 1 / (i + 1) * (
                compute_reward(action, reward_probs) - test_rewards[action][i]
            )
        else:
            test_rewards[action][i] = test_rewards[action][i - 1] + 1 / (i + 1) * (
                compute_reward(action, reward_probs) - test_rewards[action][i - 1]
            )

plt.figure(figsize=(10, 5))

for i, q in enumerate(test_rewards):
    plt.plot(q, label=f"Bandit {i+1}")

plt.legend()
plt.title(f"Average Reward Simulation ({n_sim} Runs)")
plt.show()

## 2. The Agent

### 2.1. Action-Values

**Value-based methods** (which we will focus on in this tutorial) are the most widely used in reinforcement learning. This means that the agent associates a certain value $Q$ (derived from *quality*) with each action $a$ (= choosing a bandit) and then chooses the action with the highest value (exploit) or any other action (explore). Initially, the agent has no information about which actions promise the highest reward $R$, i.e. the values are the same for all actions. After multiple interactions with the environment, the agent becomes better in estimating the true value of each action. In general, the value of an action is defined as the average, i.e. expected, reward.

$$\large Q(a) \doteq \frac{\text{sum of rewards when taken }a}{\text{number of times taken }a}$$

For repeated updating this can be written more efficiently as a weighted average of the old value and the new reward. Like this we do not have to store all values in memory, but only the old action value $Q_t(a)$ at timestep $t$ and the number of times $a$ was taken.

$$\large
Q_{t+1}(a) =  \left(1 - \frac{1}{N_{t+1}(a)}\right) Q_t(a) + \frac{1}{N_{t+1}(a)} R_{t+1}
$$

Which in turn can be transformed into an incremental averaging / updating rule:

$$\large
Q_{t+1}(a) = Q_{t}(a) + \frac{1}{N_{t+1}(a)} (R_{t+1} - Q_{t}(a))
$$

**This way of updating values is super important in reinforcement learning, so it's worth remembering.**

****: We will later replace $\frac{1}{N_{t+1}(a)}$ with the learning rate parameter $\alpha$ as this enables us to determine how strongly recent rewards should override previous value estimates. The term $R_{t+1}-Q_{t}(a)$ can also be interpreted in terms of the Rescorla-Wagner model as a prediction error (with larger errors causing larger updates). In simple terms, it is the difference between what you expect to happen (the old value) and what actually happens (the reward). If you think of it as a surprise factor, it's how surprised you are by the outcome. The greater the surprise, the more you will update your beliefs about the world (the values).

- Rescorla, R.A. & Wagner, A.R. (1972) [A theory of Pavlovian conditioning: Variations in the effectiveness of reinforcement and nonreinforcement](https://www.ualberta.ca/~egray/teaching/Rescorla%20&%20Wagner%201972.pdf), Classical Conditioning II, A.H. Black & W.F. Prokasy, Eds., pp. 64–99. Appleton-Century-Crofts.


### 2.2. The Policy

The strategy the agent uses to select actions is called **policy** (often abbreviated as $\pi$) in general. We just learned that our agent's policy is based on action values. But how exactly does the agent choose actions based on action values? One way would be to always choose the action with the highest value, the so-called **greedy policy**, but in the beginning the agent's value estimates are not very precise and a greedy policy would prevent us from exploring other potentially better actions. How much you should exploit and how much you should explore is known as the **exploration vs. exploitation dilemma**: we want to maximize reward (exploit), but we also want to minimize regret (explore).

In [ ]:
q = np.array([3, 2, 4])


def greedy_policy(q):
    max_indices = np.flatnonzero(q == q.max())
    return np.random.choice(max_indices)


for _ in range(15):
    print("Action", greedy_policy(q) + 1, "chosen")

#### 2.2.1. Epsilon-Greedy Policy

One strategy to balance exploration and exploitation is the **epsilon-greedy** policy. It works by choosing a random action with probability $\epsilon$ (exploration) and the action with the highets value with probability 1−$\epsilon$ (exploitation). This ensures that the agent explores different actions occasionally to discover potentially better options while mostly exploiting the current best-known action to maximize rewards.

Please implement the epsilon-greedy policy as a function called `eps_greedy_policy()` which takes the array of values `q`  and `epsilon` as inputs and returns an action. Do not use `np.argmax()` to select the action with the highest value (The reason is that if multiple actions have the same maximum estimated value, `np.argmax()` will consistently pick the first action with the maximum value it encounters. This leads to biased behavior and ignores other equally good options.). You can include the above `greedy_policy()` function in your function as it already takes care of this.

In [ ]:
# YOUR CODE HERE
# --------------


# --------------

In [ ]:
# TEST CELL

test_selections = np.zeros(20_000)
for i in range(test_selections.shape[0]):
    test_selections[i] = eps_greedy_policy(np.array([3, 3, 1]), 0.1)
props = (np.array([np.sum(test_selections == i) for i in range(3)]) / 20_000).round(2)
if np.all(np.isclose(props, [0.49, 0.49, 0.03], atol=0.01)):
    print("Well done! Your e-greedy function works as expected.")
else:
    raise Exception("Uh-oh! Check your code again.")

## 3. Learning from Reward

So we have initial action values and we have a policy to choose actions based on these values. But our agent still does not learn anything. This means that we keep choosing actions with the same probabilities because our action values stay the same. In the next step, we will implement that the agent receives a reward from the environment after each action and thus can update the action values (and the policy).

Here is our update rule ( that this is actually the same as equation 3 we have seen in **2.1.**):

$$\large
Q(a) \leftarrow Q(a) + \alpha (R - Q(a))
$$

$R$ is the reward the agent received after taking action $a$, and $\alpha$ (alpha) is the **learning rate**, set to values between 0 and 1. The learning rate determines the magnitude of the update, with higher values of $\alpha$ giving more weight to recent rewards.

### 3.1. Single Learning Example

In [ ]:
q = np.array([0.0, 0.0, 0.0])  # initial value estimates
alpha = 0.5  # learning rate

print("initial values:", q)

a = eps_greedy_policy(q, 0.1)
print("action taken:", a + 1)
reward = compute_reward(a, reward_probs)
print("reward:", reward)
q[a] = q[a] + alpha * (reward - q[a])  # HERE IS THE UPDATE
print("updated values:", q)

### 3.2. Learning Over Many Trials

Now let's put this all together for a simulation experiment with multiple trials. You can use a `for`-loop to iterate over `n_trials`. To analyze and visualize the learning behavior of your agent you need to store the action and the updated q array in two arrays called `action_history` and `q_history`. Take care that you initialize these arrays with the correct shape. If your code works, the following visualization should run without any changes.

In [ ]:
n_trials = 1_000  # number of trials
q = np.array([0.5, 0.5, 0.5])  # initialize starting values uniformly
alpha = 0.01  # learning rate
epsilon = 0.2  # exploration parameter

# YOUR CODE HERE
# --------------


# --------------

In [ ]:
# Plot setup
fig, ax = plt.subplots(2, 1, figsize=(9, 6))
colors = ["steelblue", "darksalmon", "teal"]

# Plot estimated values as lines
for j in range(len(q)):
    ax[0].plot(range(n_trials), q_history[:, j], color=colors[j], label=f"Action {j+1}")

# Plot actions taken as scatter
for j in range(len(q)):
    ax[1].scatter(
        np.where(action_history == j),
        action_history[action_history == j] + 1,
        color=colors[j],
        marker="x",
        s=25,
        label=f"Action {j+1}",
    )

ax[0].set_ylabel("Estimated Value")
ax[0].legend()
ax[1].set_ylabel("Action Taken")
ax[1].set_yticks(range(1, len(q) + 1))
ax[1].set_xlabel("Trial")
fig.tight_layout()
plt.show()

print("True values:", np.array(reward_probs))
print("Learned values:", q)

**Q: What do you observe here?**

## 4. Simulations

### 4.1. Exploring Different Exploration Parameters $\epsilon$

Do a simulation with 5 bandits and try at least three different parameters for $\epsilon$ (remember: they should be between 0 and 1) and choose one that works best in your simulation. Use a fixed learning rate of `alpha=0.2`, set the number of trials to `n_trials=500`, and use these reward probabilities for your reward function: `[0.1, 0.3, 0.7, 0.2, 0.1]`.

Note:

- To compare different epsilons you will need to use a performance measure such as **average reward**.   
- For a robust result you will need to run **many simulations** (`n_episodes`) each consisting of `n_trials` and average the results.  
- A good way to visualize the performance is plotting the trial number on the x-axis and the **average reward per trial** on the y-axis.  

Hint: you will need a loop for each epsilon, each episode and each trial. It makes sense to store the rewards and actions taken in arrays of shape `(len(epsilons), n_episodes, n_trials)`.

In [ ]:
n_episodes = 100
n_trials = 500  # number of trials
alpha = 0.2  # learning Srate
epsilons = [0.0, 0.1, 0.3, 0.6, 1.0]
reward_probs = [0.1, 0.3, 0.7, 0.2, 0.1]  # [0.5, 0.2, 0.0, 0.3, 0.3]

# YOUR CODE HERE
# --------------


# --------------

### 4.2. A New Policy: Softmax

Besides $\epsilon$-greedy, another solution to solve the explore-exploit tradeoff is the **softmax function with temperature parameter**. It converts action values to values between 0 and 1, so we can use them as selection probabilities. This means in practice that the agent selects the actions with the highest values most often, but also explores other actions. The amount of exploration is determined by the temperature parameter (i.e., higher temperature means more exploration).

$$\large p_i = \frac{\exp(x_i/\tau)}{\sum_{j=1}^{N}\exp(x_j/\tau)}$$


Let's define the softmax function with temperature parameter for array input ...

In [ ]:
def softmax(values, temp=1):
    """
    Calculate softmax probabilities.

    Parameters
    ----------
    values : array-like
        Input values.
    temp : float, optional
        Temperature parameter (default is 1).

    Returns
    -------
    probabilities : ndarray
        Softmax probabilities.
    """
    return np.exp(values / temp) / np.sum(np.exp(values / temp))

... and compare different temperature parameters for given action-values $Q(a)$:

In [ ]:
q = np.array([5, 4, 3])
temps = [0.5, 1, 4]

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
for i, temp in enumerate(temps):
    ps = softmax(q, temp)
    for j in range(3):
        axs[i].bar(j, ps[j])
    axs[i].set_title(f"Temperature = {temp}")
    axs[i].set_xlabel("Action")
    axs[i].set_xticks(range(n_bandit), range(1, n_bandit + 1))
    if i == 0:
        axs[i].set_ylabel("Selection Probability")
    axs[i].set_ylim(0, 1)

fig.suptitle(
    f"Action Values $Q$: ({', '.join(q.astype(str))})",
)
fig.tight_layout()

Now we are ready to implement our policy, i.e. choosing the action based on the action values. First create an array `q` for our initial estimated action values, make them equal for all three actions, then compute the choice probabilities from these values using the `softmax` function. Once you have the probabilities, you can use `np.random.choice()` to choose an action. To make the action selection procedure reusable, put this all into a function called `softmax_policy()` (which takes `q` as input).

In [ ]:
# YOUR CODE HERE
# --------------


# --------------

In [ ]:
# TEST CELL

test_selections = []
for _ in range(100):
    test_selections.append(softmax_policy(np.array([1, 1, 1]), 1))

if set(test_selections) == {0, 1, 2}:
    print("Well done! Your policy function is ready to be used!")
else:
    raise Exception("Uh-oh! Please check your code again!")

### 4.3. Exploring Different Temperature Parameters $\tau$

Now do the same simulation as before (you can re-use your code) with the $\epsilon$-greedy policy, but now try at least three different values for the **temperature parameter** and choose one that works best in your simulation. Use a fixed learning rate of `alpha=0.2`, set the number of trials to `n_trials=500`, and use these reward probabilities for your reward function: `[0.1, 0.3, 0.7, 0.2, 0.1]`.

In [ ]:
n_episodes = 100
n_trials = 500  # number of trials
alpha = 0.2  # learning rate
temps = [0.1, 0.3, 1, 3, 8]
reward_probs = [0.1, 0.3, 0.7, 0.2, 0.1]  # [0.5, 0.2, 0.0, 0.3, 0.3]

# YOUR CODE HERE
# --------------


# --------------

## 5. Non-Stationarity

So far, we have explored two different algorithms to tackle the explore-exploit tradeoff and found parameters that work well for our given problem. But we only had stationary scenarios, i.e. where the reward probabilities stayed the same. What if the reward probabilities change and a different bandit becomes the best action in the middle of an episode? How would our agents react, would they be able to adapt to this new situation?

Run the same simulations as before, but this time we want to

- compare our two policies - epsilon-greedy vs. softmax  
- compare at least three different values for the learning rate `alpha` (to see which one works best for the two algorithms)  
- keep the epsilon and temperature parameters at the values that we found worked best.  


In [ ]:
# YOUR CODE HERE
# --------------


# --------------

**Q: What do you observe?**


## 6. Model Experimental Data


In this part, we model humans in a decision task using a softmax policy and want to see if younger participants tend to explore more than older ones. Remember that in the softmax policy this is defined by the temperature parameter $\tau$.

To do this, we first create fake data for you in which human participants had to perform a task with 5 bandits. We do this by using our softmax policy and adding some noise. After you run below cell `data` is created. Inspect it to get a feel for it.

In [ ]:
reward_probs = [0.1, 0.3, 0.7, 0.2, 0.1]


def simulate_participant(
    n_trials, k_actions=5, tau=1.0, alpha=0.1, reward_probs=reward_probs
):
    values = np.zeros(k_actions)
    action_history = []
    reward_history = []
    for _ in range(n_trials):
        probs = softmax(values, tau)
        action = np.random.choice(k_actions, p=probs)
        reward = compute_reward(action, reward_probs)
        action_history.append(action)
        reward_history.append(reward)
        values[action] += alpha * (reward - values[action])
        # Add some noise to the values
        values += np.random.normal(0, 0.1, k_actions)
    return action_history, reward_history


np.random.seed(0)
n_trials = 100
participants = 100
data = []
ages = np.random.randint(12, 50, participants)

for age in ages:
    tau = 1.5 if age < 18 else 0.5
    action_history, reward_history = simulate_participant(
        n_trials, k_actions=5, tau=tau, reward_probs=reward_probs
    )
    data.append(
        {
            "age": age,
            "action_history": action_history,
            "reward_history": reward_history,
            "tau": tau,
        }
    )

For fitting the model, we will use a simple optimization approach to find the best $\tau$ that matches the observed choices, minimizing the negative log-likelihood of the observed choices given the model. We provide you with the function below and calculate tau for one example participant.

In [ ]:
from scipy.optimize import minimize

k_actions = 5

participant = 0


def rl_fit_log_l(pars, args):
    k_actions = 5
    alpha = 0.10
    tau = pars[0]
    actions, rewards = args[0], args[1]
    n_trials = len(actions)
    likelihood_tr = np.zeros(n_trials)
    values = np.zeros(k_actions)
    for i in range(n_trials):
        action_probs = softmax(values, tau)
        likelihood_tr[i] = action_probs[actions[i]]
        values[actions[i]] = values[actions[i]] + alpha * (
            rewards[i] - values[actions[i]]
        )
    return -np.sum(np.log(likelihood_tr))


action_history = data[participant]["action_history"]
reward_history = data[participant]["reward_history"]
real_tau = data[participant]["tau"]
result = minimize(
    rl_fit_log_l,
    [1.0],
    args=[action_history, reward_history],
    bounds=[(0.01, 10)],
    method="L-BFGS-B",
)
estimated_tau = result.x[0]
print(f"Real tau for participant {participant}:      {real_tau:.2f}")
print(f"Estimated tau for participant {participant}: {estimated_tau:.2f}")

Now compute $\tau$ for all participants. Divide the data into two groups of participants who are 18 or younger and participants who are older than 18. Compute the average $\tau$ in both groups and run a $t$-test to see if the difference in exploration behavior is significant. You can create a pandas DataFrame from the `data` dictionary and store the `estimated_tau` values in an own column.

In [ ]:
# YOUR CODE HERE
# --------------


# --------------

**Q: What is your conclusion?**

---
In this tutorial, we have explored bandit algorithms. They capture the essence of the exploration-exploitation trade-off in reinforcement learning (RL) and serve as a powerful tool for immediate decision-making, especially in single-state environments where the focus is on selecting actions to maximize immediate rewards.

Imagine that instead of bandits you now have routes from home to university. One of these routes is the fastest, if everything is running optimally, e.g. no trams or subways are out of service (state 1). If there is only this one state, it is a normal bandit problem, but if we have several states (one could look like this: it is 2 a.m. and only the nightline buses are running), then we are dealing with a complete RL problem with multiple state transitions, where dynamic and sequential decision-making becomes key.

---